<H1>ФБ-25 Слобода Ірина</H1>
<h2>Лабораторна робота 2</h2>

In [1]:
import datetime
import os
import urllib
from prettytable import PrettyTable
import pandas as pd
import glob


print("Setup ok")

Setup ok


Функція для завантаження даних

In [11]:
import os
import urllib.request
import datetime

def download_vhi(province_id, start_year=1981, end_year=2024):

    folder = 'data'
    os.makedirs(folder, exist_ok=True)
    
    existing = [f for f in os.listdir(folder) if f.startswith(f'vhi_id__{province_id}__')]
    if existing:
        print(f" Файл '{existing[0]}' вже є. Пропускаємо завантаження.")
        return
    
    url = (f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?"
           f"country=UKR&provinceID={province_id}&year1={start_year}&year2={end_year}&type=Mean")
    
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read()
            
            
            timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
            filepath = os.path.join(folder, f'vhi_id__{province_id}__{timestamp}.csv')
            
            with open(filepath, 'wb') as file:
                file.write(content)
                
            print(f" Файл збережено: {filepath}")
    
    except urllib.error.URLError as e:
        print(f" Помилка завантаження: {e.reason}")

# Виклик функції
#download_vhi(i)


Завантаження тестових структуровних данних за номером області

In [12]:
for i in range(1,28):
 download_vhi(i)

 Файл 'vhi_id__1__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__2__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__3__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__4__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__5__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__6__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__7__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__8__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__9__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__10__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__11__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__12__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__13__2025-03-07_10-52.csv' вже є. Пропускаємо завантаження.
 Файл 'vhi_id__14__2025-03-07_10-52.csv' вже є.

Відповідність id регіону та його назви

In [13]:
reg_id_name = {
    1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
    6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
    11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
    16: 'Рівненська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
    21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
}

Функція очищення даних і створення загального DataFrame


In [14]:
import glob
import pandas as pd

def create_data_frame(folder_path):
    # Збираємо всі CSV файли з папки
    csv_files = glob.glob(f"{folder_path}/*.csv")

    # Визначаємо заголовки стовпців
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    frames = []

    # Перебір кожного файлу
    for file in csv_files:
        # Отримуємо ID регіону з назви файлу
        region_id = int(file.split('__')[1])

        # Завантажуємо дані з CSV файлу
        df = pd.read_csv(file, header=1, names=headers)

        # Обрізаємо частину року
        df.at[0, 'Year'] = df.at[0, 'Year'][9:]

        # Видаляємо останній рядок
        df = df.drop(df.index[-1])

        # Видаляємо рядки з VHI == -1
        df = df[df['VHI'] != -1]

        # Видаляємо стовпець 'empty'
        df = df.drop(columns=['empty'])

        # Додаємо стовпець 'region_id' на початок
        df.insert(0, 'region_id', region_id)

        # Перетворюємо стовпець 'Week' на ціле число
        df['Week'] = df['Week'].astype(int)

        # Додаємо поточний датафрейм у список
        frames.append(df)

    # Об'єднуємо всі фрейми, видаляємо дублікати та скидаємо індекси
    result = pd.concat(frames).drop_duplicates().reset_index(drop=True)

    # Фільтруємо та замінюємо значення для 'region_id'
    result = result[(result['region_id'] != 12) & (result['region_id'] != 20)]
    result['region_id'] = result['region_id'].replace({
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21,
        11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17,
        22: 18, 23: 6, 24: 1, 25: 2, 26: 6, 27: 5
    })

    return result


In [15]:
resultdf = create_data_frame('data')

print(resultdf)

       region_id  Year  Week    SMN     SMT    VCI    TCI    VHI
0             21  1982     1  0.059  258.24  51.11  48.78  49.95
1             21  1982     2  0.063  261.53  55.89  38.20  47.04
2             21  1982     3  0.063  263.45  57.30  32.69  44.99
3             21  1982     4  0.061  265.10  53.96  28.62  41.29
4             21  1982     5  0.058  266.42  46.87  28.57  37.72
...          ...   ...   ...    ...     ...    ...    ...    ...
59017         20  2024    48  0.135  278.17  55.23  11.86  33.55
59018         20  2024    49  0.133  277.08  57.71  10.86  34.29
59019         20  2024    50  0.130  276.49  59.45   8.68  34.07
59020         20  2024    51  0.128  276.45  62.53   5.55  34.04
59021         20  2024    52  0.129  276.48  66.13   3.71  34.92

[54650 rows x 8 columns]


In [16]:
def region_year_analysis(region_id, years=("1982", "2024"), resultdf=None, reg_id_name=None):
    if resultdf is None or reg_id_name is None:
        raise ValueError("Both 'resultdf' and 'reg_id_name' must be provided")

    # Фільтруємо дані для заданого регіону та років
    df2 = resultdf[(resultdf["Year"].between(years[0], years[1])) & (resultdf['region_id'] == region_id)]

    # Отримуємо назву регіону
    region_name = reg_id_name.get(region_id, "Unknown region")

    # Знаходимо максимальне та мінімальне значення VHI для цього регіону
    vhi_max_reg = df2["VHI"].max()
    vhi_min_reg = df2["VHI"].min()

    # Виводимо результати
    print(f"[+] {region_name}: min {vhi_min_reg} max {vhi_max_reg}")


In [17]:
for i in range(1, 26):
    region_year_analysis(i, years=("2000", "2000"), resultdf=resultdf, reg_id_name=reg_id_name)


[+] Вінницька: min 11.25 max 63.27
[+] Волинська: min 24.65 max 68.85
[+] Дніпропетровська: min 17.77 max 61.55
[+] Донецька: min 17.4 max 69.82
[+] Житомирська: min 27.46 max 66.3
[+] Закарпатська: min 20.12 max 59.59
[+] Запорізька: min nan max nan
[+] Івано-Франківська: min 18.98 max 60.12
[+] Київська: min 10.6 max 61.87
[+] Кіровоградська: min 16.84 max 60.69
[+] Луганська: min 22.92 max 74.06
[+] Львівська: min 29.53 max 69.96
[+] Миколаївська: min 16.14 max 60.4
[+] Одеська: min 17.83 max 53.87
[+] Полтавська: min 22.52 max 68.24
[+] Рівненська: min 23.86 max 67.04
[+] Сумська: min 16.81 max 66.21
[+] Тернопільська: min 24.33 max 59.97
[+] Харківська: min 9.36 max 68.45
[+] Херсонська: min 16.12 max 63.75
[+] Хмельницька: min 20.97 max 65.0
[+] Черкаська: min 10.68 max 72.05
[+] Чернівецька: min 18.83 max 60.15
[+] Чернігівська: min 15.17 max 67.16
[+] Республіка Крим: min 23.43 max 74.45


In [18]:
from prettytable import PrettyTable

def drought_years(df, percentage, type_of_drought, start_year=None, end_year=None):
    
    if start_year is not None:
        df = df[df['Year'] >= str(start_year)]  
    if end_year is not None:
        df = df[df['Year'] <= str(end_year)]  

    
    drought_count = {}

    
    grouped_data = df.groupby('Year')

    
    for year, group_df in grouped_data:
    
        
        total_areas = len(group_df['region_id'].unique())

        if type_of_drought == "extreme":
            
            drought_areas = group_df[group_df['VHI'] < 15]['region_id'].nunique()
        elif type_of_drought == "moderate":
            
            drought_areas = group_df[(group_df['VHI'] > 15) & (group_df['VHI'] < 35)]['region_id'].nunique()
        
        
        if drought_areas / total_areas > percentage / 100:
            drought_count[year] = (drought_areas, round((drought_areas / total_areas) * 100, 2))

    
    table = PrettyTable()
    if type_of_drought == "extreme":
        table.title = f"Роки з екстремальною посухою (де більше {percentage}% областей мають VHI < 15)"
    elif type_of_drought == "moderate":
        table.title = f"Роки з помірною посухою (де більше {percentage}% областей мають VHI від 15 до 35)"

    table.field_names = ["Year", "К-сть областей з посухою", "Відсоток"]

    for year, (drought_areas, percent) in drought_count.items():
        table.add_row([year, drought_areas, f"{percent}%"])

    return table


drought_years_extreme = drought_years(resultdf, 10, "extreme", start_year='2000', end_year='2024')
print(drought_years_extreme)

drought_years_moderate = drought_years(resultdf, 10, "moderate", start_year='2000', end_year='2024')
print(drought_years_moderate)


+----------------------------------------------------------------------+
| Роки з екстремальною посухою (де більше 10% областей мають VHI < 15) |
+--------+-------------------------------------------+-----------------+
|  Year  |          К-сть областей з посухою         |     Відсоток    |
+--------+-------------------------------------------+-----------------+
|  2000  |                     4                     |      16.67%     |
|  2007  |                     5                     |      20.83%     |
+--------+-------------------------------------------+-----------------+
+-------------------------------------------------------------------------+
| Роки з помірною посухою (де більше 10% областей мають VHI від 15 до 35) |
+---------+---------------------------------------------+-----------------+
|   Year  |           К-сть областей з посухою          |     Відсоток    |
+---------+---------------------------------------------+-----------------+
|   2000  |                      24 